In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions
import boto3
import sys
import re
from datetime import datetime
import configparser
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1673844540205_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1673844540205_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1673844540205_0003,pyspark,idle,Link,Link,✔


In [10]:
Q400_128 = spark.sql('''
                            SELECT *,
        
                                CASE
                                
                                    WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 1 THEN CONCAT('00000',CAST(`Time-Per-Sec` as String)) 
                                    
                                    WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 2 THEN CONCAT('0000',CAST(`Time-Per-Sec` as String)) 
                                    
                                    WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 3 THEN CONCAT('000',CAST(`Time-Per-Sec` as String))
                                    
                                    WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 4 THEN CONCAT('00',CAST(`Time-Per-Sec` as String)) 
                                    
                                    WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 5 THEN CONCAT('0',CAST(`Time-Per-Sec` as String))
                                    
                                    ELSE `Time-Per-Sec`
                                    
                                END As Time
                                
                            FROM precogdb.q400128wps
                            
                            WHERE Filename = 'ALL ac=VT-SUG dep=HYD arr=GOI fn=SEJ4001 dt=20221121.csv'
                            
                            ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
Q400_128.createOrReplaceTempView('Q400_128')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
Q400_128_1 = Q400_128.withColumn("Time-Per-Sec-1", regexp_replace(col("Time") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
Q400_128_1.createOrReplaceTempView('Q400_128_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Q400_128_2 = Q400_128_1.withColumn("Time-1", regexp_replace(col("Time") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Q400_128_1.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Q400_512_3 = Q400_128_2.withColumn('LengthOfTimePerSec', length('Time-Per-Sec'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.sql('''
                
                SELECT Distinct(length(`time-per-sec`)),filename
                FROM 
                (
                        SELECT `time-in-secs`,`time-per-sec`,filename from precogdb.q400128wps
                        group by `time-in-secs`,`time-per-sec`, filename
                        order by `time-in-secs` ASC
                )
                where length(`time-per-sec`) in ('1','2')

 

''').show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------------------------------------------+
|length(time-per-sec)|filename                                                    |
+--------------------+------------------------------------------------------------+
|2                   |ALL ac=VT-SUQ dep=AMD arr=GOI fn=SG 2977 dt=20220323.csv    |
|2                   |ALL _1_ ac=VT-SUQ dep=JAI arr=PNQ fn=SG 2964 dt=20220329.csv|
|1                   |ALL ac=VT-SUE dep=BOM arr=HYD fn=SEJ7667 dt=20220512.csv    |
|2                   |ALL ac=VT-SUF dep=JAI arr=PNQ fn=SEJ2964 dt=20220404.csv    |
|2                   |ALL ac=VT-SUL dep=JAI arr=PNQ fn=SEJ2964 dt=20220522.csv    |
|1                   |ALL ac=VT-SUB dep=BOM arr=BLR fn=SEJ7667 dt=20211223.csv    |
|2                   |ALL ac=VT-SUH dep=BLR arr=BOM fn=SEJ7446 dt=20211106.csv    |
|2                   |ALL ac=VT-SUO dep=HYD arr=KQH fn=SG 1006 dt=20220414.csv    |
|1                   |ALL ac=VT-SUG dep=JAI arr=PNQ fn=SG 2964 dt=20220328.c

In [ ]:
#Q400_128_2.show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Q400_test = spark.sql('''
                                SELECT a.*, b.`time-per-sec` as `time-per-sec-1`
                                
                                from precogdb.q400128wps a
                                
                                CROSS JOIN
                                
                                (
                                
                                SELECT filename, `time-per-sec` from precogdb.q400128wps
                                
                                WHERE `time-in-secs` = 0
                                
                                ) b
                                
                                on a.filename = b.filename
                            


''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Q400_test_1 = Q400_test.withColumn("time-per-sec-2", regexp_replace(col("time-per-sec-1") ,  "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" )) \
                    #.withColumn("time-per-sec-2", regexp_replace(col("time-per-sec-1") ,  "(\\d{1})(\\d{2})(\\d{2})" , "$1:$2:$3" )) 
                    #.withColumn("time-per-sec-2", regexp_replace(col("time-per-sec-1") ,  "(\\d{0})(\\d{2})(\\d{2})" , "$1:$2:$3" )) \
                    #.withColumn("time-per-sec-2", regexp_replace(col("time-per-sec-1") ,  "(\\d{0})(\\d{1})(\\d{2})" , "$1:$2:$3" )) \
#createOrReplaceTempView('Q400_Test')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Q400_test_1.createOrReplaceTempView('Q400_test_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.sql('''

                SELECT `time-per-sec-1`, `time-per-sec-2`, filename
                
                FROM Q400_test_1
                
                Group by `time-per-sec-1`,`time-per-sec-2`, filename

''').show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+------------------------------------------------------------+
|time-per-sec-1|time-per-sec-2|filename                                                    |
+--------------+--------------+------------------------------------------------------------+
|150018        |1:50:018      |ALL ac=VT-SUO dep=HYD arr=GOI fn=SG 3736 dt=20220930.csv    |
|143153        |1:43:153      |ALL ac=VT-SUF dep=GOI arr=STV fn=SG 3426 dt=20220622.csv    |
|70951         |7:09:51       |ALL ac=VT-SUJ dep=JAI arr=JSA fn=SG 2975 dt=20220305.csv    |
|140520        |1:40:520      |ALL ac=VT-SUO dep=DED arr=DEL fn=SG 2034 dt=20211213.csv    |
|80557         |8:05:57       |ALL ac=VT-SUQ dep=JSA arr=AMD fn=SG 4021 dt=20221101.csv    |
|112039        |1:12:039      |ALL ac=VT-SUQ dep=DEL arr=VNS fn=SG 3741 dt=20220210.csv    |
|34612         |3:46:12       |ALL ac=VT-SUQ dep=GWL arr=BLR fn=SG 3326 dt=20220225.csv    |
|123058        |1:23:058      |ALL _1_ ac=VT-SUQ dep=CCU arr=IXS fn=SG

In [ ]:
#Q400_test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
Q400_128WPS_1 = spark.sql('''


                        SELECT
                        
                        ECN,
                        
                        UTC_TIMEDATE,
                        
                        FileName,

                        CAST((MAX(`Time-In-Secs`) - MIN(`Time-In-Secs`)) / 60 AS INT) AS FlightTime
             
                        FROM precogdb.q400128wps
                        
                        WHERE FlightDate > '2022-10-30'
                        
                        AND `Radio-Altitude` NOT LIKE '%*%'
                        
                        AND `Radio-Altitude` <> ''
                        
                        AND `Main-WOW` = 'Air (0)'
                        
                        GROUP BY ECN, UTC_TIMEDATE, Filename
                        
                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
Q400_128WPS_1.createOrReplaceTempView('Q400_128WPS_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
Q400_128WPS_1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------------+--------------------+----------+
|   ECN|       UTC_TIMEDATE|            FileName|FlightTime|
+------+-------------------+--------------------+----------+
|115066|2022-11-21 23:52:46|ALL ac=VT-SUG dep...|        62|
|121202|2022-11-21 23:52:46|ALL ac=VT-SUG dep...|        62|
+------+-------------------+--------------------+----------+

In [5]:
Q400_Time = spark.sql('''

                                SELECT CAST((MAX(CAST(`Time-in-secs` AS INT)) - MIN(CAST(`Time-in-secs` AS INT))) / 60 AS INT) AS BlockTime_DFDR, filename
                                
                                FROM precogdb.q400128wps
                                
                                WHERE FlightDate > '2022-10-30'
                                
                                
                                
                                group by filename



''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
Q400_Time.createOrReplaceTempView('Q400_Time')#count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
Q400_128WPS_2 = spark.sql('''
                        SELECT 
                        
                            DISTINCT
                        
                               Cast(ChocksOff as Date) AS FlightDate,
                               
                               TailNo,
                               
                               Departure,
                               
                               ChocksOff,
                               
                               date_format(ChocksOff, 'HH:mm:ss') AS Actual_DEP_Time,
                               
                               Arrival,
                               
                               date_format(ChocksOn, 'HH:mm:ss') AS Actual_ARR_Time,
                               
                               (bigint(to_timestamp(ChocksOn)) - bigint(to_timestamp(ChocksOff))) / 60 AS BlockTime_ARMS, 
                               
                               CONCAT(Departure, '-', Arrival) AS Sector,
                                                    
                               FlightNo,
                               
                               Date,
                               
                               ECN,
                               
                               UTC_TIMEDATE,
                               
                               CAST(SUM((CAST (`fuel-flow-no-2-kg` AS INT) + CAST (`fuel-flow-no-1-kg` AS INT))) OVER (PARTITION BY ECN, UTC_TIMEDATE, Filename) / 3600 AS INT) AS FuelConsumed,
                               
                               
                               FileName,
                               
                               loadeddate,
                               
                               year,
                               
                               month
                               
                               
                                                   
                                                   
                               
                               
                        
                        FROM precogdb.q400128wps
                        
                        WHERE FlightDate > '2022-10-30'
                        
                        AND `Radio-Altitude` NOT LIKE '%*%'
                        
                        AND `Radio-Altitude` <> ''
                        
                        AND UTC_TIMEDATE BETWEEN (ChocksOff - INTERVAL 360 MINUTES) AND (ChocksOff - INTERVAL 300 MINUTES)
                        
                        
             
                        
                        
                        
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
Q400_128WPS_2.createOrReplaceTempView('Q400_128WPS_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
Q400_128WPS_2.show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+---------+-------------------+---------------+-------+---------------+--------------+-------+--------+----------+------+-------------------+------------+--------------------------------------------------------+-----------------------+----+--------+
|FlightDate|TailNo|Departure|ChocksOff          |Actual_DEP_Time|Arrival|Actual_ARR_Time|BlockTime_ARMS|Sector |FlightNo|Date      |ECN   |UTC_TIMEDATE       |FuelConsumed|FileName                                                |loadeddate             |year|month   |
+----------+------+---------+-------------------+---------------+-------+---------------+--------------+-------+--------+----------+------+-------------------+------------+--------------------------------------------------------+-----------------------+----+--------+
|2022-11-22|VT-SUG|HYD      |2022-11-22 05:10:00|05:10:00       |GOI    |06:35:00       |85.0          |HYD-GOI|SG 4001 |2022-11-21|115066|2022-11-21 23:52:46|1064        |ALL ac=VT-SUG dep=HYD ar

In [42]:
Q400_128WPS_2.show(2, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+---------+---------------+-------+---------------+--------------+-------+--------+----------+------+-------------------+------------+--------------------------------------------------------+-----------------------+----+--------+
|FlightDate|TailNo|Departure|Actual_DEP_Time|Arrival|Actual_ARR_Time|BlockTime_ARMS|Sector |FlightNo|Date      |ECN   |UTC_TIMEDATE       |FuelConsumed|FileName                                                |loadeddate             |year|month   |
+----------+------+---------+---------------+-------+---------------+--------------+-------+--------+----------+------+-------------------+------------+--------------------------------------------------------+-----------------------+----+--------+
|2022-11-22|VT-SUG|HYD      |05:10:00       |GOI    |06:35:00       |85.0          |HYD-GOI|SG 4001 |2022-11-21|115066|2022-11-21 23:52:46|1064        |ALL ac=VT-SUG dep=HYD arr=GOI fn=SEJ4001 dt=20221121.csv|2022-12-22 08:41:35.171|2022|November|
|2022-11

In [9]:
Q400_128WPS_2_1 = spark.sql('''

                        SELECT a.*,b.Blocktime_DFDR
                        
                        FROM Q400_128WPS_2 a 
                        
                        left JOIN Q400_Time b
                        
                        ON a.filename = b.filename



''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
Q400_128WPS_2_1.createOrReplaceTempView('Q400_128WPS_2_1')#show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
Q400_FOB_1 = spark.sql('''

                    SELECT 
                    
                    b.flightdate,
                               
                               b.TailNo,
                               
                               b.Departure,
                               
                               b.Actual_DEP_Time,
                               
                               b.Arrival,
                               
                               b.Sector,
                               
                               b.Blocktime_ARMS,
                               
                               b.Blocktime_DFDR,
                               
                               a.FlightTime,
                                                    
                               b.FlightNo,
                               
                               b.Date,
                               
                               b.FuelConsumed,
                               
                               b.FileName,
                               
                               b.loadeddate,
                               
                               b.year,
                               
                               b.month
                               
                               FROM Q400_128WPS_1 a
                               
                               INNER JOIN Q400_128WPS_2_1 b
                               
                               on a.filename = b.filename
                               
                               and a.ECN = b.ECN
                               
                               and a.UTC_TIMEDATE = b.UTC_TIMEDATE
                               
                               
                               
                               
                               

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
from pyspark.sql import functions as F
Q400_FOB = Q400_FOB_1.withColumn('Blocktime_ARMS', F.expr('concat(div(CAST(Blocktime_ARMS AS INT), 60), ":", mod(CAST(Blocktime_ARMS AS INT), 60))'))\
                        .withColumn('Blocktime_DFDR', F.expr('concat(div(CAST(Blocktime_DFDR AS INT), 60), ":", mod(CAST(Blocktime_DFDR AS INT), 60))'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
Q400_FOB.createOrReplaceTempView('Q400_FOB')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
spark.sql('''

                SELECT * FROM Q400_FOB
                
                WHERE Filename = 'ALL ac=VT-SUG dep=HYD arr=GOI fn=SEJ4001 dt=20221121.csv'

''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+---------+---------------+-------+-------+--------------+--------------+----------+--------+----------+------------+--------------------+--------------------+----+--------+
|flightdate|TailNo|Departure|Actual_DEP_Time|Arrival| Sector|Blocktime_ARMS|Blocktime_DFDR|FlightTime|FlightNo|      Date|FuelConsumed|            FileName|          loadeddate|year|   month|
+----------+------+---------+---------------+-------+-------+--------------+--------------+----------+--------+----------+------------+--------------------+--------------------+----+--------+
|2022-11-22|VT-SUG|      HYD|       05:10:00|    GOI|HYD-GOI|          1:25|          1:18|        62| SG 4001|2022-11-21|        1064|ALL ac=VT-SUG dep...|2022-12-22 08:41:...|2022|November|
+----------+------+---------+---------------+-------+-------+--------------+--------------+----------+--------+----------+------------+--------------------+--------------------+----+--------+

In [14]:
Q400_FOB.createOrReplaceTempView('Q400_FOB')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
jdbcDF = spark.read \
  .format("jdbc") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=SALT") \
  .option("dbtable", "LoadAndTrimDetails") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .load()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
jdbcDF.createOrReplaceTempView('jdbcDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
loadDF = spark.sql('''

                SELECT a.flightdate,
                               
                               a.TailNo,
                               
                               a.FlightNo,
                               
                               a.Sector,
                               
                               a.Departure,
                               
                               a.Actual_DEP_Time,
                               
                               a.Arrival,
                               
                               a.Actual_ARR_Time,
                               
                               b.FOB,
                               
                               (CAST (b.FOB AS INT) + 105) AS DepartureFuel,
                               
                               a.FuelConsumed,
                               
                               a.Blocktime_ARMS,
                               
                               a.Blocktime_DFDR,
                               
                               a.FlightTime,
                                                    
                               ((CAST (b.FOB AS INT) + 105) - CAST (a.FuelConsumed AS INT)) AS ArrivalFuel,
                               
                               a.FileName,
                               
                               a.loadeddate,
                               
                               a.year,
                               
                               a.month,
                               
                               current_timestamp()   AS `ProcessedDate`
                               
                               
                
                FROM Q400_FOB a 
                
                LEFT JOIN jdbcDF b on a.FlightDate = CAST(b.FlightDate AS date)
                
                AND REPLACE(a.FlightNo,' ','') = b.FlightNo
                
                AND a.Departure = b.Source
                
                AND a.Arrival = b.Destination
                
                
                                
                
                


''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
cannot resolve '`a.Actual_ARR_Time`' given input columns: [b.AcftPurpose, b.AcftRegn, b.ActCabStr, b.ActCompStr, b.ActLoadDistStr, b.ActLoadDistStrV2, b.ActcrewStr, a.Actual_DEP_Time, b.AdjustStr, b.AdjustStrv2, b.AdultLDM, a.Arrival, b.BagLDM, a.Blocktime_ARMS, a.Blocktime_DFDR, b.CabStr, b.CaptEmpId, b.Captain, b.CargoLDM, b.CargoOnSeatAuto, b.CargoOnSeatStr, b.CompStr, b.CrewStr, a.Date, a.Departure, b.Destination, b.DeviationTOWMAC, b.EDNO, b.EZFW, b.EZFWupTime, b.FOB, a.FileName, b.FlightDate, a.FlightNo, b.FlightNo, a.FlightTime, a.FuelConsumed, b.Id, b.InfantLDM, b.LPLoginID, b.LTLoginID, b.LW, b.LWMAC, b.LWMACAFT, b.LWMACFWD, b.LWindex, b.LastCreatedTime, b.LastUpdateOn, b.LastUpdatedTime, b.LoadDistStr, b.LoadDistStrV2, b.LoadOfficer, b.LoadedBy, b.LoadedOn, b.OEW, b.OEW_Index, b.OLW, b.OTOW, b.OZFW, b.PalletsStr, b.PalletsStrAuto, b.RTOW, a.Sector, b.SectorLDM, b.Source, b.SpecialStr, b.TOW, b.TOWMAC, b.TOWMACAFT, b.TOWMACFWD, b.TOWindex, a.TailNo, b

In [13]:
loadDF.write.mode('append').partitionBy("year", "month").parquet('s3://sjet-datamart-bucket/FuelOnBoard/fuel-on-board-processed-q400/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
loadDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- flightdate: date (nullable = true)
 |-- TailNo: string (nullable = true)
 |-- FlightNo: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Departure: string (nullable = true)
 |-- Actual_DEP_Time: string (nullable = true)
 |-- Arrival: string (nullable = true)
 |-- Actual_ARR_Time: string (nullable = true)
 |-- FOB: integer (nullable = true)
 |-- DepartureFuel: integer (nullable = true)
 |-- FuelConsumed: integer (nullable = true)
 |-- Blocktime_ARMS: string (nullable = true)
 |-- Blocktime_DFDR: string (nullable = true)
 |-- FlightTime: integer (nullable = true)
 |-- ArrivalFuel: integer (nullable = true)
 |-- FileName: string (nullable = true)
 |-- loadeddate: timestamp (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)

In [24]:
loadDF \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=precog_rnd") \
  .option("dbtable", "FuelOnBoard_Q400") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
loadDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183

In [ ]:
spark.stop()